<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_%EB%B3%80%ED%98%95_%EB%93%80%EC%96%BC%EB%AA%A8%EB%A9%98%ED%85%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [2]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 13.2 MB/s eta 0:00:00


In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
start_date = today - timedelta(days=375)

# 전략명
str_strategy = '변형 듀얼모멘텀 '

# 전략에 사용할 ETF 명 리스트에 담기
etf_aggressive = ['SPY', 'EFA']
etf_defensive = ['SHY', 'IEF', 'TLT', 'TIP', 'LQD', 'HYG', 'BWX', 'EMB']
etf_bil = ['BIL']

# 자산군 리스트 더하기, 중복 제거
etf_order = etf_aggressive + etf_defensive + etf_bil

# ETF 가격 추출
df_etf = yf.download(etf_order, start=start_date, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0)

display(df_etf)

[*********************100%%**********************]  11 of 11 completed


,SPY,EFA,SHY,IEF,TLT,TIP,LQD,HYG,BWX,EMB,BIL
2023-09-01,451.1900,71.4900,80.9900,93.9800,94.8500,105.2000,105.0300,75.0700,21.9700,85.1500,91.4700
2023-08-31,450.3500,71.5300,81.0340,94.5580,96.3510,105.7470,105.7830,74.9290,22.1280,85.3610,91.4550
2023-08-30,451.0100,71.8100,80.9542,94.3685,95.9223,105.5277,105.6036,74.9788,22.0881,85.4407,91.3953
2023-08-29,449.1600,71.8500,80.9542,94.3785,96.0220,105.6872,105.7531,75.0086,22.1280,85.6697,91.3754
2023-08-28,442.7600,70.9700,80.7747,93.7102,95.0350,105.1688,104.9161,74.5408,21.9083,84.7934,91.3853
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-30,391.8118,60.7582,80.1046,98.5379,109.3000,110.8473,105.8963,70.4004,22.0039,81.4957,87.7216
2022-08-29,396.1608,61.1983,80.1436,98.5670,109.0291,111.0402,105.7621,70.9453,22.0138,81.5241,87.7120
2022-08-26,398.7977,61.3352,80.2020,99.1309,109.9386,111.4743,106.5484,71.2177,22.1613,82.0170,87.7120
2022-08-25,412.7696,62.9882,80.2215,99.2475,109.1161,111.6382,107.1909,72.4108,22.3382,83.3154,87.7120


In [4]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} 전통 듀얼모멘텀.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')


# 월평균영업일수. average Monthly Business Day
MBD = 21

# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-09-03 변형 듀얼모멘텀 ### ETF 현재-1-3-6-12 개월 가격

,SPY,EFA,SHY,IEF,TLT,TIP,LQD,HYG,BWX,EMB,BIL
today/2023-09-01,451.1900,71.4900,80.9900,93.9800,94.8500,105.2000,105.0300,75.0700,21.9700,85.1500,91.4700
1mo_ago/2023-08-03,448.8400,71.9600,80.7048,93.8099,94.5664,105.1289,104.4378,73.8640,22.2478,84.7536,91.0767
3mo_ago/2023-06-02,426.3363,71.3422,80.7558,96.5937,101.1316,106.5795,106.2201,73.4837,22.3543,83.8327,90.2976
6mo_ago/2023-03-03,401.1631,69.6041,79.5867,93.8245,100.2568,105.9737,104.0726,72.6237,21.9214,83.2054,89.2725
12mo_ago/2022-08-31,388.8305,60.3474,80.0949,98.1296,108.2550,109.8636,104.7840,70.0152,21.8860,80.8418,87.7408


In [5]:
## DataFrame 만들기: 수익률
ror_1mo = df_etf.pct_change(-MBD).iloc[0:1]*100    # 1개월 수익률
ror_3mo = df_etf.pct_change(-MBD*3).iloc[0:1]*100    # 3개월 수익률
ror_6mo = df_etf.pct_change(-MBD*6).iloc[0:1]*100   # 6개월 수익률
ror_12mo = df_etf.pct_change(-MBD*12).iloc[0:1]*100  # 12개월 수익률

df_etf_ror = pd.concat([ror_1mo, ror_3mo, ror_6mo, ror_12mo])
df_etf_ror.index = df_momentum_index[1:]

str_etf_ror = str_today + str_strategy + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)

### 2023-09-03 변형 듀얼모멘텀 ### ETF 1-3-6-12 개월 수익률

,SPY,EFA,SHY,IEF,TLT,TIP,LQD,HYG,BWX,EMB,BIL
1mo_ago/2023-08-03,0.5236,-0.6531,0.3533,0.1813,0.2999,0.0676,0.5670,1.6327,-1.2488,0.4678,0.4319
3mo_ago/2023-06-02,5.8296,0.2071,0.2900,-2.7058,-6.2114,-1.2943,-1.1204,2.1588,-1.7190,1.5714,1.2983
6mo_ago/2023-03-03,12.4705,2.7095,1.7632,0.1657,-5.3930,-0.7301,0.9199,3.3684,0.2217,2.3371,2.4615
12mo_ago/2022-08-31,16.0377,18.4641,1.1175,-4.2286,-12.3828,-4.2449,0.2348,7.2195,0.3840,5.3292,4.2503


In [7]:
# SPY, EFA 수익이 둘 다 BIL 보다 낮으면 채권에 투자
str_is_bond = 'False'
if ror_12mo['SPY'].iloc[0] < ror_12mo['BIL'].iloc[0] and ror_12mo['EFA'].iloc[0] < ror_12mo['BIL'].iloc[0]:
  str_is_bond = 'True'

# 채권 중 수익률 상위 3개에 각 33.3% 씩 투자! 그 중 6개월 수익률이 -이면 현금 보유
str_bond_top_3 = str_today + str_strategy + '### 6개월 수익률 안전자산 상위 3개'
ma_6mo_defensive_top_3 = ror_6mo[etf_defensive].transpose().nlargest(3, str(df_etf.index[0].date())).transpose()
display(Markdown(str_bond_top_3))
display(ma_6mo_defensive_top_3)

# 안전자산 중 0 이상인 것만 필터링하기
defensive_etf_list_text = ''

if str_is_bond == 'True':
  filtered_index = ma_6mo_defensive_top_3.columns[(ma_6mo_defensive_top_3 < 0).any()]
  df_etf_defensive_positive_index = ma_6mo_defensive_top_3.drop(filtered_index, axis=1).columns
  defensive_etf_list_text = ' '.join([str(asset) for asset in df_etf_defensive_positive_index])


## 결과 출력
display(Markdown(str_today[1:] + str_strategy))

str_etf = "## 12개월 수익률 기준, 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? " + str_is_bond
display(Markdown(str_etf))

# 아니라면 SPY, EFA 중 12개월 수익률이 더 높은 것 표시
str_higher_stock = ''

if str_is_bond == 'False':
  if ror_12mo['SPY'].iloc[0] > ror_12mo['EFA'].iloc[0]:
    str_higher_stock = 'SPY'
  else:
    str_higher_stock = 'EFA'

str_higher_stock_text = "## 투자할 공격자산: " + str_higher_stock
display(Markdown(str_higher_stock_text))

# 안전자산 중 6개월 수익률 0 이상 출력
str_etf_defensive_positive = '## 투자할 안전자산: ' + defensive_etf_list_text
display(Markdown(str_etf_defensive_positive))

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_etf}\n')
  f.write(f'{str_higher_stock_text}\n')

### 2023-09-03 변형 듀얼모멘텀 ### 6개월 수익률 안전자산 상위 3개

,HYG,EMB,SHY
2023-09-01,3.3684,2.3371,1.7632


## 2023-09-03 변형 듀얼모멘텀 

## 12개월 수익률 기준, 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? False

## 투자할 공격자산: EFA

## 투자할 안전자산: 